# Modelo de Recompensa ou Reward Model (RM)

O modelo de recompensa é treinado para aprender uma função escalar que estima o quão boa é uma resposta 
𝑦 para um prompt x, de acordo com um critério de preferência (humano ou de IA).

O RM normalmente é treinado com perguntas e pares de respostas escolhidas e rejeitadas, esse tipo de dataset é chamado de "preferências" em que podem ser gerados tanto por humanos como por IA.

Normalmente, o RM é um LLM pré-treinado com uma camada linear que transforma o hidden state do último token em um score escalar.

# UltraFeedback

Como já existe um RM treinado pela equipe que desenvolveu o artigo do Ultrafeedback, vamos apenas importar o modelo do Hugging Face para ver como ele funciona.

https://huggingface.co/openbmb/UltraRM-13b

In [ ]:
from transformers import PreTrainedModel, LlamaConfig, LlamaModel, LlamaTokenizer
import torch.nn as nn
import torch
from typing import Optional, List

class LlamaRewardModel(PreTrainedModel):
    config_class = LlamaConfig
    def __init__(self, config):
        super().__init__(config)
        self.model = LlamaModel(config)
        self.regression_head = nn.Linear(self.config.hidden_size, 1, bias=False)

    def forward( # args are the same as LlamaForCausalLM
        self,
        input_ids: torch.LongTensor = None,
        attention_mask: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
        past_key_values: Optional[List[torch.FloatTensor]] = None,
        inputs_embeds: Optional[torch.FloatTensor] = None,
        labels: Optional[torch.LongTensor] = None,
        use_cache: Optional[bool] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ):

        transformer_outputs = self.model(
                                input_ids,
                                attention_mask=attention_mask,
                                position_ids=position_ids,
                                past_key_values=past_key_values,
                                inputs_embeds=inputs_embeds,                               
                            )

        hidden_states = transformer_outputs[0]
        rewards = self.regression_head(hidden_states).squeeze(-1)
        
        ends = attention_mask.cumsum(dim=1).argmax(dim=1).view(-1,1)
        rewards = torch.gather(rewards, 1, ends)
        
        return rewards

ultrarm_template = """Human: {instruction}

Assistant: {completion}"""

dataset = [ # cases in webgpt
  {"chosen":"Human: \"Who orders martinis \"\"shaken, not stirred\"\"?\"\nAssistant: Sean Connery's character, fictional British Secret Service agent James Bond, in the movie Goldfinger, stated that he preferred his martini to be \"shaken, not stirred\". [1] Some believe that Bond ordered his martini shaken because of the vodka it contained, as vodka was, for the most part, refined from potatoes (cheaper brands) which made the vodka oily. To disperse the oil, Bond ordered his martinis shaken. [2]","rejected":"Human: \"Who orders martinis \"\"shaken, not stirred\"\"?\"\nAssistant: Fleming's fictional British Secret Service agent James Bond orders his martini cocktail shaken, not stirred [1]. Bond's preferences for his martini are carried over to the films, where his orders are seen in both the 1961 film Dr. No and the 2006 film Casino Royale [1, 2]. In both films, Bond's subordinates copy his order, telling the bartender to keep the fruit with their drinks [2]. However, in the 2006 film, Bond appears irritated when the bartender asks if he would like his drink shaken or stirred [2]."},
  {"chosen":"Human: Sural relates to which part of the body?\nAssistant: The sural region is the muscular swelling of the back of the leg below the knee, formed chiefly by the bellies of the gastrocnemius and soleus muscles [1,2].","rejected":"Human: Sural relates to which part of the body?\nAssistant: The Sural nerve runs down the side of the leg near the small saphenous vein, then passes forward below the lateral malleolus and continues on the outside of the foot as the lateral dorsal cutaneous nerve, which then communicates with the intermediate dorsal cutaneous nerve, which branches off to the side of the foot. [1]"}
]

tokenizer = LlamaTokenizer.from_pretrained("openbmb/UltraRM-13b")
model = LlamaRewardModel.from_pretrained("openbmb/UltraRM-13b")
model.eval()

for example in dataset:
    inputs = tokenizer(example["chosen"], return_tensors="pt")
    chosen_reward = model(**inputs).item()
    inputs = tokenizer(example["rejected"], return_tensors="pt")
    rejected_reward = model(**inputs).item()
    print(chosen_reward - rejected_reward)

# Output 1: 2.4158712085336447
# Output 2: 0.1896953582763672


# Sem LLM

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# ============================================================
# 1. Pequeno "dataset" simulado
# ============================================================
dataset = [
    {
        "pergunta": "Explique o que é aprendizado por reforço.",
        "chosen":   "É um método de aprendizado em que um agente aprende por tentativa e erro, recebendo recompensas por boas ações.",
        "rejected": "É uma técnica onde o modelo reforça os erros para aprender mais rápido."
    },
    {
        "pergunta": "O que é uma rede neural?",
        "chosen":   "É um modelo computacional inspirado no cérebro humano, usado para reconhecer padrões e tomar decisões.",
        "rejected": "É uma ferramenta que copia o cérebro humano literalmente e pensa sozinha."
    }
]

# ============================================================
# 2. Tokenização bem simples (só contagem de palavras)
# ============================================================
def encode_text(text):
    return torch.tensor([len(text.split())], dtype=torch.float32)

# ============================================================
# 3. Modelo de recompensa simples: uma camada linear
# ============================================================
class SimpleRewardModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(1, 1)  # entrada = número de tokens, saída = score
    
    def forward(self, x):
        return self.linear(x)

model = SimpleRewardModel()
optimizer = optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.BCEWithLogitsLoss()

# ============================================================
# 4. Função de perda tipo "pairwise ranking loss"
#    (igual usada em RLHF/RLAIF)
# ============================================================
def pairwise_loss(chosen_reward, rejected_reward):
    return -torch.log(torch.sigmoid(chosen_reward - rejected_reward)).mean()

# ============================================================
# 5. Treinamento
# ============================================================
for epoch in range(100):
    total_loss = 0
    for item in dataset:
        chosen_input = encode_text(item["pergunta"] + " " + item["chosen"])
        rejected_input = encode_text(item["pergunta"] + " " + item["rejected"])

        chosen_reward = model(chosen_input)
        rejected_reward = model(rejected_input)

        loss = pairwise_loss(chosen_reward, rejected_reward)
        total_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if epoch % 20 == 0:
        print(f"Época {epoch} | Perda média: {total_loss / len(dataset):.4f}")

# ============================================================
# 6. Avaliação: ver se o modelo aprendeu a dar maior score ao chosen
# ============================================================
for item in dataset:
    chosen_input = encode_text(item["pergunta"] + " " + item["chosen"])
    rejected_input = encode_text(item["pergunta"] + " " + item["rejected"])
    with torch.no_grad():
        r_chosen = model(chosen_input).item()
        r_rejected = model(rejected_input).item()
    print(f"\nPergunta: {item['pergunta'], item['chosen'], item['rejected']}")
    print(f"Score chosen  : {r_chosen:.4f}")
    print(f"Score rejected: {r_rejected:.4f}")


Época 0 | Perda média: 0.1413
Época 20 | Perda média: 0.0489
Época 40 | Perda média: 0.0266
Época 60 | Perda média: 0.0176
Época 80 | Perda média: 0.0128

Pergunta: ('Explique o que é aprendizado por reforço.', 'É um método de aprendizado em que um agente aprende por tentativa e erro, recebendo recompensas por boas ações.', 'É uma técnica onde o modelo reforça os erros para aprender mais rápido.')
Score chosen  : 33.5429
Score rejected: 25.6767

Pergunta: ('O que é uma rede neural?', 'É um modelo computacional inspirado no cérebro humano, usado para reconhecer padrões e tomar decisões.', 'É uma ferramenta que copia o cérebro humano literalmente e pensa sozinha.')
Score chosen  : 26.9877
Score rejected: 23.0547
